In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
! kaggle competitions download -c convolve-epoch1

 80% 53.0M/65.9M [00:00<00:00, 275MB/s]
100% 65.9M/65.9M [00:00<00:00, 272MB/s]


In [ ]:
! mkdir data
! unzip convolve-epoch1 -d data

Archive:  convolve-epoch1.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.json         


In [ ]:
import json
with open("./data/train.json", 'r') as f:
      datastore = json.load(f)
print(len(datastore))

4152659


In [ ]:
import random

# step =0
ana = []
for i in datastore:
  if datastore[i] == "abnormal":
    res = i.split(' ', 5)[5]
    res = res.rsplit(' ',1)[0]
    ana.append((res,datastore[i]))

print(len(ana))

nor = []
for i in datastore:
  if datastore[i] == "normal":
    temp = i.split(' ',6)[6]
    nor.append((temp,datastore[i]))

print(len(nor))

# 4082967

data = random.sample(ana,10000)
data+= random.sample(nor, 60000)

random.shuffle(data)
val = 0
for i in data:
  if(i[1] == 'abnormal'):
    print("hi")

In [ ]:
for i in datastore:
  if(datastore[i] == "abnormal"):
    res = i.split(' ', 5)[5]
    print(res,datastore[i])
    break
for i in datastore:
  if(datastore[i] == "normal"):
    temp = i.split(' ', 6)[6]
    print(temp,datastore[i])
    break

RAS APP FATAL ciod: failed to read message prefix on control stream (CioStream socket to 172.16.96.116:33399 abnormal
RAS KERNEL INFO instruction cache parity error corrected
 normal


In [ ]:
#Initialize the lists
sentences = []
labels = []

# Collect sentences and labels into the lists
for item in data:
    sentences.append(item[0])
    if item[1] == "normal":
      labels.append(0)
    else:
      labels.append(1)

In [ ]:
training_size = int(0.9*len(data))
valid_size = len(data) - training_size

# Vocabulary size of the tokenizer
vocab_size = 1000000

# Maximum length of the padded sequences
max_length = 200

# Output dimensions of the Embedding layer
embedding_dim = 32

training_sentences = sentences[:training_size]
training_labels = labels[:training_size]

valid_sentences = sentences[training_size:]
valid_labels = labels[training_size:]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters for padding and OOV tokens
trunc_type='pre'
padding_type='post'
oov_tok = "<OOV>"
# print(training_sentences[0])
# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# Generate the word index dictionary
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Generate and pad the testing sequences
valid_sequences = tokenizer.texts_to_sequences(valid_sentences)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert the labels lists into numpy arrays
training_labels = np.array(training_labels)
valid_labels = np.array(valid_labels)

# print(training_padded[0])

In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

num_epochs = 3

# Train the model
model.fit(training_padded, training_labels, epochs=num_epochs,steps_per_epoch=100, validation_data=(valid_padded, valid_labels), verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           32000000  
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                792       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 32,000,817
Trainable params: 32,000,817
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
100/100 - 48s - loss: 0.4844 - accuracy: 0.9739 - val_loss: 0.2199 - val_accuracy: 0.9838 - 48s/epoc

In [ ]:
import csv

testing = []
with open('./data/test.csv','r') as csvfile:
  csvreader = csv.reader(csvfile, delimiter=',')
  next(csvreader)
  for i in csvreader:
    testing.append(i[1].rstrip())

In [ ]:
testing_s = tokenizer.texts_to_sequences(testing)
testing_padded = pad_sequences(testing_s, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
print(len(testing_padded))
print(len(testing))
print(testing[0])

595300
595300
 1124336301 2005.08.17 R13-M1-N8-C:J12-U01 2005-08-17-20.38.21.466368 R13-M1-N8-C:J12-U01 RAS KERNEL FATAL rts: kernel terminated for reason 1001


In [ ]:
res = model.predict(testing_padded)

with open('submission.csv','w') as csvfile:
  # creating a csv writer object
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(['ID',' Label'])
  for i in range(len(res)):
    if res[i] > 0.5:
      t = "abnormal"
    else:
      t = "normal"
    csvwriter.writerow([i,t])

print(len(res))
# step = 0
# for i in res:
#   if i[0] > 0.5:
#     print(step, i[0])
#     break
#   step+=1



18604/18604 [==============================] - 34s 2ms/step
595300


In [ ]:
! kaggle competitions submit -c convolve-epoch1 -f submission.csv -m "Message"

100% 8.41M/8.41M [00:01<00:00, 4.51MB/s]
Successfully submitted to Convolve Epoch-1